In [1]:
from itertools import permutations

In [1]:
def leitura_fasta_q(fasta_q):
    'extrai as sequencias de um arquivo fastaq'
    reads = []  
    with open (fasta_q) as fq:     
        while True:
            fq.readline() #skip line
            sequencia =  fq.readline().rstrip()
            fq.readline() #skip
            qualidade= fq.readline().rstrip()
            if len(sequencia) == 0 or len(qualidade) == 0:
                break
            reads.append(sequencia)
    return reads

In [2]:
def get_kmers(read, n):
    'subdivide o texto em strings de tamanho k'
    kmers = []
    for i in range (len(read) - n + 1):
        kmers.append([read[i:i + n],i])
    return kmers
            

In [3]:
def find_overlap(sufix,prefix,n):
    '''encontra overlaps de tamanho n entre o sufixo e o prefixo 
       Ex -  overlap de tamanho 4:
           CGCGAAGT (sufixo)
               AAGTCCCCCAAA (prefixo)
    '''
    start = 0
    while True:
        start =  sufix.find(prefix[:n], start)
        
        if start == -1: # nao encontrou overlap de tamanho n
            return 0
        
        if prefix.startswith(sufix[start:]):
            return len(sufix) - start
        
        start += 1

In [4]:
#obtem as sequencias do fastaq
reads =  leitura_fasta_q('ERR266411_1.for_asm.fastq')

In [5]:
#Para cada read, obtem os kmers de tamanho 6 e adiciona ao dicionario de kmers
kmer_reads = {}
n = 6 

print('Inicio')
#associa as reads a cada kmer
for read in reads:
    kmers = get_kmers(read, 6)
    for kmer in kmers:
        # inicializa a key no dict, se nao existir
        if  kmer[0] not in kmer_reads.keys():
            kmer_reads[kmer[0]] = set()
            
        #adiciona a read    
        kmer_reads[kmer[0]].add(read)
print('Tamanho kmer_reads: ', len(kmer_reads))

Inicio
Tamanho kmer_reads:  4594


In [6]:
print(len(kmer_reads['TAAACA']))

256


In [7]:
#para cada read, pega o sufixo e busca no dicionario kmer_reads as outras reads que tem esse kmer
overlap_pairs = set()
overlap_by_read = {}
for read in reads:
    start = len(read) - n
    sufixo = read[start:]
    reads_com_sufixo = kmer_reads[sufixo]
    overlap_by_read[read] = 0
    for r in reads_com_sufixo:
        if r != read:
            overlap = find_overlap(read, r, 30)
            if overlap > 0 :
                overlap_pairs.add((read, r))
                overlap_by_read[read] += 1

In [9]:
print(len(overlap_pairs))

904746


In [10]:
#print(overlap_by_read['AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTCTG'])

In [11]:
#obtem quantos sufixos tiveram overlaps
sufixos_com_overlaps =0 
for i, j in overlap_by_read.items():
    if j > 0:
        sufixos_com_overlaps += 1
        
print(sufixos_com_overlaps)

7161
